## **Importing Dependencies**

In [1]:
import os
import SimpleITK as sitk
import numpy as np

## **Data Directories**

In [2]:
train_adc_dir = r"E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Train\1ADC_ss"
train_z_adc_dir = r"E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Train\2Z_ADC"
train_label_dir = r"E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Train\3LABEL"
val_adc_dir = r"E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Val\1ADC_ss"
val_z_adc_dir = r"E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Val\2Z_ADC"
val_label_dir = r"E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Val\3LABEL"
preprocessed_dir = r"E:\University\Masters\Semester 2\DL\Term Project\Data\Preprocessed"
train_output_dir = os.path.join(preprocessed_dir, "Train")
val_output_dir = os.path.join(preprocessed_dir, "Val")

## **Exploring Map and Mask Sizes**

In [3]:
# List of directories to scan
directories = [
    train_adc_dir,
    train_z_adc_dir,
    train_label_dir,
    val_adc_dir,
    val_z_adc_dir,
    val_label_dir,
]

In [4]:
# Function to calculate the average, maximum, and minimum size from a directory
def calculate_sizes(directory):
    total_size = [0, 0, 0] 
    max_size = [0, 0, 0]  
    min_size = [float("inf"), float("inf"), float("inf")]  
    file_count = 0 

    for file_name in os.listdir(directory):
        if file_name.endswith(".mha"):
            file_path = os.path.join(directory, file_name)
            try:
                image = sitk.ReadImage(file_path)
                size = image.GetSize() 
                total_size = [t + s for t, s in zip(total_size, size)]
                max_size = [max(m, s) for m, s in zip(max_size, size)]
                min_size = [min(m, s) for m, s in zip(min_size, size)]
                file_count += 1
            except Exception as e:
                print(f"Error reading file {file_path}: {e}")

    if file_count > 0:
        average_size = tuple(t // file_count for t in total_size)
        return average_size, tuple(max_size), tuple(min_size)
    else:
        return (0, 0, 0), (0, 0, 0), (0, 0, 0)

In [5]:
# Iterate over all directories and find sizes
overall_max_average_size = (0, 0, 0)
overall_max_size = (0, 0, 0)
overall_min_size = (float("inf"), float("inf"), float("inf"))

for dir_path in directories:
    dir_average_size, dir_max_size, dir_min_size = calculate_sizes(dir_path)
    overall_max_average_size = tuple(max(o, a) for o, a in zip(overall_max_average_size, dir_average_size))
    overall_max_size = tuple(max(o, m) for o, m in zip(overall_max_size, dir_max_size))
    overall_min_size = tuple(min(o, m) for o, m in zip(overall_min_size, dir_min_size))
    
    print(f"Directory: {dir_path}")
    print(f"  Average size: {dir_average_size}")
    print(f"  Maximum size: {dir_max_size}")
    print(f"  Minimum size: {dir_min_size}")

Directory: E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Train\1ADC_ss
  Average size: (176, 176, 37)
  Maximum size: (256, 256, 64)
  Minimum size: (128, 128, 16)
Directory: E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Train\2Z_ADC
  Average size: (176, 176, 37)
  Maximum size: (256, 256, 64)
  Minimum size: (128, 128, 16)
Directory: E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Train\3LABEL
  Average size: (176, 176, 37)
  Maximum size: (256, 256, 64)
  Minimum size: (128, 128, 16)
Directory: E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Val\1ADC_ss
  Average size: (136, 132, 27)
  Maximum size: (160, 144, 51)
  Minimum size: (128, 128, 16)
Directory: E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Val\2Z_ADC
  Average size: (136, 132, 27)
  Maximum size: (160, 144, 51)
  Minimum size: (128, 128, 16)
Directory: E:\University\Masters\Semester 2\DL\Term Project\Data\BONBID2023_Val\3LABEL
  Ave

In [6]:
print(f"\nOverall maximum average size across all directories: {overall_max_average_size}")
print(f"Overall maximum size across all directories: {overall_max_size}")
print(f"Overall minimum size across all directories: {overall_min_size}")


Overall maximum average size across all directories: (176, 176, 37)
Overall maximum size across all directories: (256, 256, 64)
Overall minimum size across all directories: (128, 128, 16)


## **Resampling Maps and Masks**
* 3D medical images are memory intensive and so to reduce computational overhead during training, the data is preprocessed offline as doing it on-the-fly can lead to duplicate memory allocation.
* First preprocessing step is to resample maps and masks to a fixed size to make Pytorch batching possible and help model learn efficiently without being biased by size variations. Maps and labels are resampled to (H,W,D) -> (192,192,32) which is maximum of average dimension across all directories, rounded to a multiple of 2^4=16 (total downsampling factor of traditional UNet) to ensure clean downsampling and upsampling (without fractional sizes that require inconsistent padding). For maps resizing is done using Trilinear interpolation because it provides smooth transitions in intensity values preserving the overall structure without introducing artifacts and for mask Nearest Neighbor interpolation is used because it preserves discrete class boundaries, preventing the creation of intermediate invalid label values. For example, 0.5 for a binary mask.
We didn't resample to maximum dimension across all directories purely due to memory constraints as otherwise we believe it to be the superior choice and we didn't resample to minimum dimension across all directories as it would lead to too much data loss. Current choice strikes a balance between preserving structural detail and memory efficiency.
* In the second step, the resampled (resized) maps' intensities are normalized individually using mean and standard deviation to make the model more robust by mitigating differences in scanners and patient anatomy (along with the first step above).
* Lastly, both adc and z_adc maps are concatenated (stacked on top of each other) to make a 2-channel input (C,H,W,D) -> (2,192,192,32) for UNet to allow the model to leverage information from both modalities.

In [4]:
# Create the output directories
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)
os.makedirs(os.path.join(train_output_dir, "MAP"), exist_ok=True)
os.makedirs(os.path.join(train_output_dir, "LABEL"), exist_ok=True)
os.makedirs(os.path.join(val_output_dir, "MAP"), exist_ok=True)
os.makedirs(os.path.join(val_output_dir, "LABEL"), exist_ok=True)

# Metadata dictionary
metadata = {}

# Helper function to load .mha files
def load_mha_file(file_path):
    img = sitk.ReadImage(file_path)
    img_array = sitk.GetArrayFromImage(img)
    return img_array, img

# Helper Function to normalize image intensity using mean and standard deviation
def normalize_intensity(img_array):
    mean = np.mean(img_array)
    std = np.std(img_array)
    return (img_array - mean) / std

# Helper Function to save processed .mha images
def save_mha_image(img, output_path):
    sitk.WriteImage(img, output_path)

# Helper function to resample images to a fixed size
def resample_image(image, target_size=(192, 192, 32), is_mask=False): # SimpleITK convention (x,y,z) -> (W,H,D)
    original_size = image.GetSize() 
    original_spacing = image.GetSpacing()  
    
    # Compute new spacing based on target size
    new_spacing = [
        original_spacing[i] * (original_size[i] / target_size[i])
        for i in range(3)
    ]
    
    # Set up resampler
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(target_size)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    
    # Use nearest neighbor for binary masks, linear interpolation otherwise
    if is_mask:
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)

    # Apply resizing
    resized_image = resampler.Execute(image)
    return resized_image

# Function to process the dataset
def process_dataset(adc_dir, z_adc_dir, label_dir, output_dir, dataset_name):
    adc_files = {file_name.replace("-ADC_ss.mha", "") for file_name in os.listdir(adc_dir)}
    z_adc_files = {file_name.replace("Zmap_", "").replace("-ADC_smooth2mm_clipped10.mha", "") for file_name in os.listdir(z_adc_dir)}
    label_files = {file_name.replace("_lesion.mha", "") for file_name in os.listdir(label_dir)}

    common_files = adc_files.intersection(z_adc_files).intersection(label_files)
    print(f"Common files for {dataset_name}: {len(common_files)}")

    for base_name in common_files:
        adc_path = os.path.join(adc_dir, f"{base_name}-ADC_ss.mha")
        z_adc_path = os.path.join(z_adc_dir, f"Zmap_{base_name}-ADC_smooth2mm_clipped10.mha")
        label_path = os.path.join(label_dir, f"{base_name}_lesion.mha")

        # Load images
        _, adc_sitk = load_mha_file(adc_path)
        _, z_adc_sitk = load_mha_file(z_adc_path)
        _, label_sitk = load_mha_file(label_path)
        print('Original ADC Shape', sitk.GetArrayFromImage(adc_sitk).shape)
        print('Original Z_ADC Shape', sitk.GetArrayFromImage(z_adc_sitk).shape)
        print('Original Label Shape', sitk.GetArrayFromImage(label_sitk).shape)

        # Resample images
        adc_resampled = resample_image(adc_sitk, is_mask=False)
        z_adc_resampled = resample_image(z_adc_sitk, is_mask=False)

        # Normalize intensity for ADC and Z-ADC
        adc_resampled_array = normalize_intensity(sitk.GetArrayFromImage(adc_resampled))
        z_adc_resampled_array = normalize_intensity(sitk.GetArrayFromImage(z_adc_resampled))

        # Resample binary mask (label)
        label_resampled = resample_image(label_sitk, is_mask=True)

        # Concatenate along channel axis (C, D, H, W)
        concatenated_map = np.stack((adc_resampled_array, z_adc_resampled_array), axis=0)
        print('Concatenated Map Shape', concatenated_map.shape)
        print('Resampled Label Shape', sitk.GetArrayFromImage(label_resampled).shape)
        concatenated_map_sitk = sitk.GetImageFromArray(concatenated_map)

        # Save MAP and label
        save_mha_image(concatenated_map_sitk, os.path.join(output_dir, "MAP", f"{base_name}_concatenated.mha"))
        save_mha_image(label_resampled, os.path.join(output_dir, "LABEL", f"{base_name}_lesion.mha"))
        print(f"Processed and saved for {base_name} in {dataset_name}")

# Process datasets
process_dataset(train_adc_dir, train_z_adc_dir, train_label_dir, train_output_dir, "Train")
process_dataset(val_adc_dir, val_z_adc_dir, val_label_dir, val_output_dir, "Val")

Common files for Train: 85
Original ADC Shape (57, 128, 128)
Original Z_ADC Shape (57, 128, 128)
Original Label Shape (57, 128, 128)
Concatenated Map Shape (2, 32, 192, 192)
Resampled Label Shape (32, 192, 192)
Processed and saved for MGHNICU_306-VISIT_01 in Train
Original ADC Shape (23, 128, 128)
Original Z_ADC Shape (23, 128, 128)
Original Label Shape (23, 128, 128)
Concatenated Map Shape (2, 32, 192, 192)
Resampled Label Shape (32, 192, 192)
Processed and saved for MGHNICU_010-VISIT_01 in Train
Original ADC Shape (48, 256, 256)
Original Z_ADC Shape (48, 256, 256)
Original Label Shape (48, 256, 256)
Concatenated Map Shape (2, 32, 192, 192)
Resampled Label Shape (32, 192, 192)
Processed and saved for MGHNICU_105-VISIT_01 in Train
Original ADC Shape (46, 256, 256)
Original Z_ADC Shape (46, 256, 256)
Original Label Shape (46, 256, 256)
Concatenated Map Shape (2, 32, 192, 192)
Resampled Label Shape (32, 192, 192)
Processed and saved for MGHNICU_153-VISIT_01 in Train
Original ADC Shape (2